In [1]:
# Query 
from __future__ import division
import pyodbc, sys, socket
# Web Scrappnig
import requests
from bs4 import BeautifulSoup
# Pandas
import pandas as pd
# RegEx / Regular Expressions
import re
from re import sub
# Json
import json
# Limpar Output
from IPython.display import clear_output

In [2]:
arg=socket.gethostname()
cnxn = pyodbc.connect('DRIVER={DBMAKER};SERVER=' + arg + ';DATABASE=COTRIPALDB_SIMPLUS_BASE;UID=SYSADM;PWD=')
cursor = cnxn.cursor()

sql=("select a.gtin , a.codigo_cotripal, cast(a.descricao as varchar(250)) as nome, a.id_erp_departamento, a.id_erp_setor, a.id_erp_item from produto as a where a.id_erp_departamento = 018 and a.descricao like '%VV%' ORDER BY IDX;")


cursor.execute(sql)

rows = cursor.fetchall()

cursor.close()

In [3]:
headers = {
    "User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"
}
global url
url = 'https://www.vv.com.br/pt/produto/detalhe/'
global infos
infos = []

notFound = pd.DataFrame()
notFound['gtin'] = ''
notFound['cod_cotripal'] = ''
notFound['nome_produto'] = ''
notFound['id_erp_departamento'] = ''
notFound['id_erp_setor'] = ''
notFound['id_erp_item'] = ''
notFound['info_extra'] = ''

In [4]:
def findVVCod(nome):
    indice = nome.find('VV')
    nome = nome[indice:]
    nome = nome.replace(' ', '/')
    nome = nome.split('/')
    cods = []
    for cod in nome:
        if 'VV-' in cod:
            cod = cod.replace('VV-', '')
            cod = re.sub('[A-Z]', '', cod)
            cods.append(cod)
    return cods

In [5]:
def search(row):
    cod = str(findVVCod(row[2])[0])
    if not '-' in cod:
        notFound.loc[len(notFound)] = [row[0], row[1], row[2], row[3], row[4], row[5], 'Cód. VV sem sufixo']
        return
    link = str(url + cod)
    req = requests.get(link, headers=headers)
    if req.status_code != 200:
        notFound.loc[len(notFound)] = [row[0], row[1], row[2], row[3], row[4], row[5], f'Erro na requisição: status {req.status_code}']
        return
    site = BeautifulSoup(req.text, "html.parser")
    equipamentos = site.find("ul", class_="listarEquipamentos")
    if not equipamentos:
        notFound.loc[len(notFound)] = [row[0], row[1], row[2], row[3], row[4], row[5], 'Produto não encontrato no site']
        return
    equipamentosList = []
    for li in equipamentos.contents:
        if len(li) > 1:
            keys = []
            values = []
            for el in li.find_all("h3"):
                keys.append(el.get_text())
            for el in li.find_all("p"):
                values.append(el.get_text())
            try:
                equipamentosList.append({ keys[0] : values[0], keys[1] : values[1], keys[2] : values[2:] })
            except:
                equipamentosList.append({ keys[0] : values[0], keys[1] : values[1] })
    urlImagens = []
    imagens = site.find("ul", class_="listaGaleria")
    imagens = imagens.find_all("img")
    for img in imagens:
        urlImagens.append("https://www.vv.com.br/" + img["src"])
    nome = site.find("div", class_="subtitulo")
    nome = nome.find("h2").get_text()
    descricao = site.find("div", class_="descricao")
    descricao = descricao.find("ul")
    descricao_keys = []
    descricao_values =[]
    for li in descricao.contents:
        if len(li) > 1:
            for el in li.find_all("h3"):
                descricao_keys.append(el.get_text())
            for el in li.find_all("p"):
                descricao_values.append(el.get_text())

    itemInfo = {
        "gtin" : row[0],
        "codigo_cotripal" : row[1],
        "id_erp_departamento" : row[3],
        "id_erp_setor" : row[4],
        "id_erp_item" : row[5],
        "nome" : nome,
        "imagens" : urlImagens
    }
    for i, key in enumerate(descricao_keys):
        itemInfo.update({key : descricao_values[i]})
    itemInfo.update({"Equipamentos":equipamentosList})
    infos.append(itemInfo)

In [6]:
for i, row in enumerate(rows):
    try:
        search(row)
        clear_output(wait=True)
        print('|- Carregando... {:->12}'.format('|'))
        print('|{:->28}'.format('|'))
        print('|- {:06.2f}% Concluido {:->8}'.format(((i + 1)/len(rows))*100, '|'))
        print('|- {:03} Produtos restantes {:->3}'.format(len(rows)-(i + 1), '|'))
        print('|{:->28}'.format('|'))
    except IndexError as e:
        notFound.loc[len(notFound)] = [row[0], row[1], row[2], row[3], row[4], row[5], f'Problemas em identificar o cód VV no nome']
    except Exception as e:
        notFound.loc[len(notFound)] = [row[0], row[1], row[2], row[3], row[4], row[5], f'{e}']

|- Carregando... -----------|
|---------------------------|
|- 100.00% Concluido -------|
|- 000 Produtos restantes --|
|---------------------------|


In [7]:
with open(f'VV/BeautifulSoap/infos.json', 'a') as arq:
    arq.write(json.dumps(infos, ensure_ascii=False))
notFound.to_excel(f'VV/BeautifulSoap/notFound.xlsx')

<h1><span style="color:#333333;font-weight: 400">Tempo de execução: <span style="color:#05AC4B; font-weight: 900">00:02:15</span></span></h1>
<h1><span style="color:#333333;font-weight: 400">Total de produtos: <span style="color:#2F80ED; font-weight: 900">427</span></span></h1>
    <h1><span style="color:#333333;font-weight: 400">Produtos não encontrados: <span style="color:#D40000; font-weight: 900">86</span></span></h1>
<h1><span style="color:#333333;font-weight: 400">Percentual de produtos encontrados: <span style="color:#6720C0; font-weight: 900">79.86%</span></span></h1>
